In [ ]:
import csv
#the prerequisites relationship dataset in this case is a matrix
#  B  B  B ...
#A 0  1  0
#A 0  0  1
#A 0  0  0
#..
#we don't really have 0 and 1, we have a number of votes 
#the votes are given by 4 annotators
#vote = "I think there's a prerequisite relationship"
#in this scenario we consider 2+ votes as 1, everything else as 0

#the dataframe we create
#              A   B   0/1
df = {'A':[],'B':[],'prerequisite':[]}
B_list = []

with open('/content/drive/My Drive/datasets/matrix_with_annotators_names.csv') as csv_file:
    #the first row is the header containing the B elements
    #we save it to know the relationships
    B_list = []
    is_first_line = True
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      if is_first_line:
        B_list = row
        is_first_line = False
      else:
        #the first element is the element A
        #every other row is the prerequisite relationship with B
        #so we skip the first element
        A = row[0]
        row = iter(row)
        next(row)
        for index, element in enumerate(row):
          #as said before, +2 votes = 1
          #else = 0
          df['A'].append(A)
          #index+1 because we skipped the first element
          df['B'].append(B_list[index+1])
          if len(element.split(" ")) >= 2:
            df['prerequisite'].append(1)
          else:
            df['prerequisite'].append(0)

df = pd.DataFrame.from_dict(df)

In [ ]:
#create dictionary of tokenized documents
nltk.download('punkt')
nltk.download('stopwords')

punct = string.punctuation + '«``»' + "''"
en_stop_words = nltk.corpus.stopwords.words('english')

doc_dict = {}
#this is the text file with the context information for each term
#we don't have wikipedia pages or something similar
#just this text file
#so we will get the 10 words around each term to get a context
#we get a context every time we find the term inside the file
#so if a term appears multiple times we get the context multiple times so there's more information
page_file = open('/content/drive/My Drive/datasets/cap4_annotato_PRET.txt', "r")
page = page_file.read()
page_file.close()
#we clean the text file 
page = re.sub(r'<.+?>', ' ', page)
page = page.replace("#"," ")
page = page.replace("@"," ")
#we split the string into an array of words
#since the word titles are composed by multiple words divided by space
#we first divide the string so that those words are considered a single word


for index,row in df.iterrows():
  #Titles of the two documents
  title_A = row[0]
  title_B = row[1]
  
  #we check if the page related to the titles is already in the dictionary
  if title_A not in doc_dict:
    #find the 10 words around title_A
    document = ""
    #we get the ten words around every match
    #there's a problem with this
    #some terms are substrings of other terms
    #for example COMPUTER is a substring of COMPUTER NETWORKS
    #this means COMPUTER will match with the COMPUTER of COMPUTER NETWORKS
    #I don't have the time to solve this problem
    for match in re.finditer(title_A, page):
      before = page[:match.start()].split()[15:]
      after = page[match.start():].split()[:15]
      before = ' '.join([str(elem) for elem in before]) 
      after = ' '.join([str(elem) for elem in after]) 
      document = document + before + after
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = document.replace("s","")
    document = document.replace("'"," ")
    document = document.replace("\\"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_A] = no_stop_words

  if title_B not in doc_dict:
    #find the 10 words around title_B
    document = ""
    #we get the ten words around every match
    #there's a problem with this
    #some terms are substrings of other terms
    #for example COMPUTER is a substring of COMPUTER NETWORKS
    #this means COMPUTER will match with the COMPUTER of COMPUTER NETWORKS
    #I don't have the time to solve this problem
    for match in re.finditer(title_B, page):
      before = page[:match.start()].split()[10:]
      after = page[match.start():].split()[:10]
      before = ' '.join([str(elem) for elem in before]) 
      after = ' '.join([str(elem) for elem in after]) 
      document = document + before + after
    #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
    document = document.replace("'"," ")
    document = document.replace("\\"," ")
    #tokenized
    tokenized = nltk.tokenize.word_tokenize(document, "english")
    #no punctuation and lowercase
    no_punct = [x.lower() for x in tokenized if x not in punct]
    #remove stop words
    no_stop_words = [x for x in no_punct if x not in en_stop_words]
    #add tokenized document
    doc_dict[title_B] = no_stop_words 